In [29]:
# Crop each image to the bounding box for each animal
    # and save it to the cropped_dir directory
    for i, box in enumerate(prediction["boxes"]):
        # Convert float values to integers
        box = torch.round(box)
        x1, y1, x2, y2 = box
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        print(x1, y1, x2, y2)
        cropped_im = crop(im, x1, y1, x2, y2)
        cropped_im.save(os.path.join(cropped_dir, f"{filename}_{i}.jpg"))

IndentationError: unexpected indent (2496637211.py, line 3)

In [26]:
print(start_time)

NameError: name 'start_time' is not defined